In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
gc.collect()

items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)


In [25]:
#sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
#sales_train = sales_train[sales_train['date_block_num'].isin([21,22])]
sales_train = sales_train[sales_train['date_block_num'].isin([33])]

sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [26]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
#train_item_ids = ids_keep
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

#all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_item_ids = test_item_ids

#all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))
all_shop_ids = test_shop_ids



In [27]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [28]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [29]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [30]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [31]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id"]



y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [11]:
def add_lags(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()

        df = df.merge(result, on=cols, how='left')
        df[lag_name].fillna(0,inplace=True)
        if "mean" in name:
            df[lag_name] = pd.to_numeric(df[lag_name], downcast='float')
        else:
            df[lag_name] = pd.to_numeric(df[lag_name].astype(int), downcast='unsigned')
        del result
        gc.collect()
    
    return df
                                         

                                        
training = add_lags(training, ['item_id','date_block_num'], 'item_cnt_block')
#training = add_lags(training, ['item_id','date_block_num'], 'item_cnt_block_mean')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_cnt_block')
#training = add_lags(training, ['shop_id','date_block_num'], 'shop_cnt_block_mean')
training = add_lags(training, ['item_category_id','date_block_num'], 'category_cnt_block')
#training = add_lags(training, ['item_category_id','date_block_num'], 'category_cnt_block_mean')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_category_cnt_block')
#training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_category_cnt_block_mean')

item_cnt_block 1
shop_cnt_block 1
category_cnt_block 1
shop_category_cnt_block 1


In [12]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding', 'item_cnt_block_lag_1',
       'shop_cnt_block_lag_1', 'category_cnt_block_lag_1',
       'shop_category_cnt_block_lag_1'], dtype=object)

In [33]:
all_features = [
 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding', 'item_cnt_block_lag_1',
       'shop_cnt_block_lag_1', 'category_cnt_block_lag_1',
       'shop_category_cnt_block_lag_1'
]

In [34]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 


training[all_features] = StandardScaler().fit_transform(training[all_features])
#StandardScaler().fit(training[all_features])


#training[all_features] = training[all_features].apply(pd.to_numeric, downcast='float')

In [35]:
training.fillna(0,inplace=True)

In [16]:
#training = training[training['date_block_num'].isin([32,33])]

In [36]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
8174,1132,41,33,0,55,10,2015,-0.043643,-0.607444,0.968274,-0.032129,0.002243,-0.581245,-0.142205
106319,11117,26,33,0,40,10,2015,-0.176311,-0.108340,1.671072,0.821984,-0.348508,-0.043529,-0.089922
159007,16009,55,33,0,64,10,2015,-0.030376,0.312942,-0.885580,-0.540179,0.023500,-0.114272,-0.270725
45598,4610,44,33,0,55,10,2015,0.022691,-0.579758,0.968274,0.372839,0.064835,-0.585044,-0.144406
114564,12117,46,33,0,16,10,2015,-0.176311,-0.174938,-1.187422,-0.532816,-0.348508,-0.055269,-0.295039
25168,2865,16,33,0,25,10,2015,-0.096710,-0.442074,-1.111211,-0.451822,-0.153647,-0.378814,-0.244356
162232,16125,44,33,0,64,10,2015,0.009425,-0.579758,-0.885580,-0.341377,0.248256,-0.514844,-0.259238
97646,10304,56,33,0,38,10,2015,-0.017109,-0.217590,-0.843156,-0.363466,0.135503,-0.150203,-0.129196
85304,8850,4,33,0,40,10,2015,-0.176311,-0.540847,1.671072,0.196125,-0.348508,-0.601377,-0.094127
162168,16123,10,33,0,64,10,2015,-0.123244,-0.842405,-0.885580,-0.518090,-0.178004,-0.860339,-0.267336


In [18]:
scales = dict(zip([21,22],[np.log(x) for x in [21,22]]))
print(scales)

{21: 3.044522437723423, 22: 3.091042453358316}


In [19]:
training['dbn_scaled'] = training['date_block_num'].apply(lambda dbn: scales[dbn])

In [20]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding', 'item_cnt_block_lag_1',
       'shop_cnt_block_lag_1', 'category_cnt_block_lag_1',
       'shop_category_cnt_block_lag_1', 'dbn_scaled'], dtype=object)

In [21]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor

features = [
'item_id_mean_encoding', 'shop_id_mean_encoding', 'item_category_id_mean_encoding',
    'item_cnt_block_lag_1',
       'shop_cnt_block_lag_1', 'category_cnt_block_lag_1',
       'shop_category_cnt_block_lag_1', 'dbn_scaled'
]

lr_model =  LinearRegression()
train = training[training['date_block_num']==22]
x_train = train[features]
y_train = train['shop_item_cnt_block']
lr_model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

train_preds = lr_model.predict(x_train)
train_preds.clip(0,20,out=train_preds)
rms = sqrt(mean_squared_error(y_train, train_preds))
print("rmse: ", rms)

rmse:  0.9007020386011617


In [22]:
list(zip(features,lr_model.coef_))

[('item_id_mean_encoding', 1.0161182349403604),
 ('shop_id_mean_encoding', 0.1457812840995624),
 ('item_category_id_mean_encoding', 0.08430661573847892),
 ('item_cnt_block_lag_1', -0.17789029305943854),
 ('shop_cnt_block_lag_1', -0.02709896353502962),
 ('category_cnt_block_lag_1', -0.07224189388879666),
 ('shop_category_cnt_block_lag_1', 0.09314177209902676),
 ('dbn_scaled', 0.0015124753549550498)]

In [45]:
test.head()

,ID,shop_id,item_id,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding,item_cnt_block_lag_1,shop_cnt_block_lag_1,category_cnt_block_lag_1,shop_category_cnt_block_lag_1,dbn_scaled
0,0,5,5037,0.547856,-0.278282,0.380675,0.155360,-0.375477,-0.295035,0.012049,3.526361
1,1,5,5320,-0.348508,-0.278282,-0.146701,-0.176311,-0.375477,0.968274,0.240304,3.526361
2,2,5,5233,0.895168,-0.278282,0.380675,0.380896,-0.375477,-0.295035,0.012049,3.526361
3,3,5,5232,0.636626,-0.324261,0.236785,0.195160,-0.375477,-0.385137,-0.135211,3.526361
4,4,5,5268,-0.348508,-0.319524,1.959103,-0.176311,-0.375477,1.264110,0.122495,3.526361


In [38]:
feats  = [
    'item_id_mean_encoding', 'shop_id_mean_encoding', 'item_category_id_mean_encoding',
    'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block',
       'shop_category_cnt_block'
    ]

In [39]:
cols = ['item_id', 'shop_id']

In [40]:
test = test.merge(training[training['date_block_num'] == 33][cols + feats], on=cols, how='left')

In [42]:
test.rename(columns={"item_cnt_block":"item_cnt_block_lag_1", "shop_cnt_block":"shop_cnt_block_lag_1",\
                     "category_cnt_block":"category_cnt_block_lag_1", "shop_category_cnt_block":"shop_category_cnt_block_lag_1"},inplace=True)

In [ ]:
len(test)

In [44]:
test['dbn_scaled'] = np.log(34)

In [46]:
preds = lr_model.predict(test[features])
preds.clip(0,20,out=preds)

array([0.85794372, 0.        , 1.17073343, ..., 0.        , 0.        ,
       0.        ])

In [3]:

print(np.mean(preds))
print(np.max(preds))

0.264143421233
9.45581838102


In [6]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
bestpreds = pd.read_csv('submissionbest.csv')['item_cnt_month']
print(np.mean(bestpreds))
print(np.max(bestpreds))

In [2]:
lgbm_preds = pd.read_csv('submission.csv')['item_cnt_month']
lstm_preds = pd.read_csv('lstm104.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lgbm_preds * 0.5) + (lstm_preds * 0.5)